# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [2]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
# 4. Display the updated dtypes using df.dtypes
df = transform_types(df, {'enrollment_date': 'datetime'})
df = transform_types(df, {'site': 'category', 'intervention_group': 'category', 'sex': 'category'})

df = transform_types(df, {"age": "numeric", "bmi": "numeric", "systolic_bp": "numeric", "diastolic_bp": "numeric", "cholesterol_total": "numeric", "cholesterol_hdl": "numeric", "cholesterol_ldl": "numeric", "glucose_fasting": "numeric", "follow_up_months": "numeric", "adherence_pct": "numeric", "adverse_events": "numeric"})
print(df.dtypes)

patient_id                    object
age                          float64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object


## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [3]:
# TODO: Calculate cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_total'] / df['cholesterol_hdl']

In [4]:
# TODO: Categorize blood pressure

def categorize_systolic(systolic_bp):
    if systolic_bp < 120:
        return 'Normal'
    if 120 <= systolic_bp <= 129:
        return 'Elevated'
    if systolic_bp >= 130:
        return 'High'

df['bp_category'] = df['systolic_bp'].apply(categorize_systolic)
# create_bins(df, "systolic_bp", bins =[-float('inf'), 120, 129, float('inf')], labels = ['Normal', 'Elevated', 'High'], new_column = "bp_category")

**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [5]:
# TODO: Create age groups
df = create_bins(df, 'age', bins=[0, 40, 55, 70, 100], labels=['<40', '40-54', '55-69', '70+'], new_column='age_group')

In [6]:
# TODO: Create BMI categories
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    if 18.5 <= bmi <=24.9:
        return 'Normal'
    if 25 <= bmi <=29.9:
        return 'Overweight'
    if bmi >=30:
        return 'Obese'

df['bmi_category'] = df['bmi'].apply(categorize_bmi)
# create_bins(df, 'bmi', bins=[-float('inf'), 18.5, 25, 29.9, float('inf')], labels=['Underweight', 'Normal', 'Overweight', 'Obese'], new_column='bmi_category')

## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [7]:
# TODO: String cleaning
df['sex'] = df['sex'].str.strip().str.lower()
df['sex'] = df['sex'].replace({"f": "female", "m": "male"})
df["outcome_cvd"] = df["outcome_cvd"].str.strip().str.lower()
df.to_csv("output/q6_load_this_data.csv", index = False)

## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [8]:
# TODO: One-hot encoding
df = pd.get_dummies(df, columns=['intervention_group', 'site'], dtype='int64')
print(df.shape)
print(df.columns)

(10000, 28)
Index(['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp',
       'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl',
       'cholesterol_ldl', 'glucose_fasting', 'follow_up_months',
       'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout',
       'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category',
       'intervention_group_control', 'intervention_group_treatment a',
       'intervention_group_treatment b', 'site_site a', 'site_site b',
       'site_site c', 'site_site d', 'site_site e'],
      dtype='object')


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [9]:
# TODO: Save transformed data
# df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
df.to_csv('output/q6_transformed_data.csv', index=False)